_Main issue to be solved: information sources are no longer included in the links I use (from mid-2016 onwards), so the live up-to-date aspect of the app is at risk._

In [1]:
import pandas as pd
import requests
import re # for regex in html
from bs4 import BeautifulSoup
import sqlalchemy

%load_ext line_profiler

connect_param = 'postgresql://macowner@localhost:5432/macowner'
engine = sqlalchemy.create_engine(connect_param)

pd.options.display.max_columns=500
pd.options.display.max_rows=1500
pd.options.display.max_colwidth=1000

data_ = pd.read_json('https://s3.amazonaws.com/postgraphics/policeshootings/policeshootings2015.json')

In [4]:
data = data_[data_.armed=="unarmed"].copy()
data = data.reset_index()

sources = pd.DataFrame(data["sources"].apply(lambda x:[y["url"] for y in x]).tolist())

for col in sources.columns:
    data["source_"+str(col)]=sources[col]


failed_urls = []  

def scraper(URL):
    if URL==None:
        pass
        return
    else:
        try:
            r = requests.get(URL) 
        except Exception:
            failed_urls.append(URL) 
            # http://www.huffingtonpost.com/2015/03/05/derek-cruice-shot-in-face_n_6805902.html failed. check why.
            pass
            return
            
        soup = BeautifulSoup(r.content, "lxml") # Warning message advises specifying BeautifulSoup(YOUR_MARKUP, "lxml")
    
        return soup.prettify()
    
df2015_with_info = data.copy() # for reference, df2015_with_info.shape is (94, 30)


for source_col in ['source_0','source_1','source_2','source_3','source_4']:
    df2015_with_info[source_col] = df2015_with_info[source_col].apply(scraper)

In [5]:
data_with = df2015_with_info

In [1]:
# only one that failed in 2015 data.
# got html manually.

missing = ("""
<div class="entry__text js-entry-text bn-entry-text" data-beacon="{&quot;p&quot;:{&quot;mlid&quot;:&quot;entry_text&quot;}}" data-beacon-parsed="true">
<p>A Volusia County Sheriff’s deputy shot an unarmed man in the face while serving a drug warrant on Wednesday.</p>
<p>Deputies say Derek Cruice, 26, resisted arrest during the incident at his Deltona, Florida home. Cruice’s friends, six of whom were at the house when a SWAT team entered, called the fatal shooting “murder.”</p>
<p>At 6:32 a.m., Deputy Todd Raible, 36, <a href="http://www.wftv.com/news/news/local/officials-man-shot-face-while-deputies-attempt-ser/nkNWP/" rel="nofollow" target="_hplink" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;fired one shot&quot;,&quot;mpid&quot;:1,&quot;plid&quot;:&quot;http://www.wftv.com/news/news/local/officials-man-shot-face-while-deputies-attempt-ser/nkNWP/&quot;}}" data-beacon-parsed="true" class="bn-clickable">fired one shot</a>, hitting Cruice in the face as he stood inside the doorway, police told WFTV. </p>
<p>Cruice was pronounced dead a short time later at an area hospital.</p>
<p>“They were met with resistance and a shooting incurred,” Volusia County Sheriff Ben Johnson told My News 13.</p>
<p>The sheriff’s department <a href="http://www.mynews13.com/content/news/cfnews13/news/article.html/content/news/articles/cfn/2015/3/4/man_shot_by_volusia_.html" rel="nofollow" target="_hplink" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;confirmed that Cruice was unarmed&quot;,&quot;mpid&quot;:2,&quot;plid&quot;:&quot;http://www.mynews13.com/content/news/cfnews13/news/article.html/content/news/articles/cfn/2015/3/4/man_shot_by_volusia_.html&quot;}}" data-beacon-parsed="true" class="bn-clickable">confirmed that Cruice was unarmed</a> at the time.</p>
<blockquote class="twitter-tweet twitter-tweet-error" lang="en" data-twitter-extracted-i1498835941005278827="true"></blockquote>
<p>Deputy involved in shooting while serving warrant in Deltona. Suspect shot once in face after advancing on deputy. No injury to deputies.</p>
<p>— Volusia Co. Sheriff (@VolusiaSheriff) <a href="https://twitter.com/VolusiaSheriff/status/573110522876452864" rel="nofollow" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;March 4, 2015&quot;,&quot;mpid&quot;:3,&quot;plid&quot;:&quot;https://twitter.com/VolusiaSheriff/status/573110522876452864&quot;}}" data-beacon-parsed="true" class="bn-clickable">March 4, 2015</a></p>

<script async="" src="//platform.twitter.com/widgets.js" charset="utf-8">try{}catch(e){}</script>
<p>One of the men inside the home, 24-year-old Matthew Grady, contradicted deputies’ claims that Cruice had resisted arrest.</p>
<p>“There’s a couple of seconds between opening the door, walking out, getting to my knee and halfway out there’s gunfire,” Grady told My News 13. “I look back as the guy’s grabbing me, and my friend is dead or dying.”</p>
<p>When asked if Cruice resisted, Grady said: “No.”</p>
<p>Roommate Steven Cochran said Cruice wasn’t wearing a shirt at the time of the shooting, and didn’t pose a threat of carrying a concealed weapon.</p>
<p>“It’s kind of hard to conceal anything or hide anything when this is all you have on,” said Cochran. “They entered the house and fired.”</p>
<p>Raible, a SWAT team member and narcotics investigator with the department for 10 years, reportedly has a clean record. He was placed on administrative leave, and the Florida Department of Law Enforcement has taken over the investigation. Raible wasn’t injured in the incident, <a href="https://twitter.com/VolusiaSheriff/status/573110522876452864" rel="nofollow" target="_hplink" data-beacon="{&quot;p&quot;:{&quot;lnid&quot;:&quot;according to the sheriff’s department&quot;,&quot;mpid&quot;:4,&quot;plid&quot;:&quot;https://twitter.com/VolusiaSheriff/status/573110522876452864&quot;}}" data-beacon-parsed="true" class="bn-clickable">according to the sheriff’s department</a>.</p>
<p>The sheriff’s office allegedly recovered 217 grams of marijuana at the home, along with a scale, pipes, plastic bags and about $3,000 in cash.</p>
<p>Cruice had previously been arrested on misdemeanor drug charges in the past, but was never convicted, WFTV reported.</p>

<script class="entry-paragraph-1-ad-script-template" type="text/template">

</script>
<script class="sharebox-ad-script-template" type="text/template">

</script>
<script class="in-line-ad-unit-script-template" type="text/template">

</script>
<script class="mobileweb_transporter_mid_script-template" type="text/template">

</script>
<script class="mobileweb_transporter_mid_2_script-template" type="text/template">

</script>
<script class="entry-paragraph-3-ad-script-template" type="text/template">

</script>

</div>
""")

In [7]:
data_with.loc[20,'source_2'] = missing

In [59]:
# I dropped the columns ['sources', 'photos', 'videos'] (the 1st two being dict values) because the to_sql() method
# of the pandas Dataframe was not happy with them.

data_with.to_sql('data2015_with_html', engine, if_exists='replace')

__Extracting information from html__

In [9]:
#searching the pages (mainly the article) for information

def extractor(article, term_to_search):

    if (article!= None) & (type(article)!= float): #just for NaNs
    
        #removing paragraph tags
    
        article=article.replace("<p>"," ")
        article=re.sub("<p class=\"paragraph\"[^>]*>" , " ", article)
        article=article.replace("</p>"," ")

        #finding article text (and related text) 
        my_regex = '[^>]*' + re.escape(term_to_search) + '[^<]*'
        #this searches text contained between the tag symbols '>' & '<' for the term_to_search
        #re.escape is to escape any special characters
        

        information=""
        for match in re.findall(my_regex, article, re.DOTALL): # the
            if ("{" not in match) & ("=" not in match):
                information+=match
            else:
                pass
        if len(information)==0:
            return "No match."
        else:
            return information
    
    else:
        return "No html, or is NaN value."

In [41]:
data_with_2 = data_with.copy() # for use with extractor() funtion

# this uses just the name of the victim. Trying with other info could/should increase yield
for source_col in ['source_0','source_1','source_2','source_3','source_4']:
    for ind, name in zip(data_with_2.index,data_with_2.name):
        data_with_2.loc[ind,source_col] = extractor(data_with_2.loc[ind,source_col],name)

I wrote the following in my report:

"The name of the dead was not always present in at least one of the news articles reporting it, but this was just to obtain extra information: all cases have at least a one to two description with some relevant information."

__Profiler__

In [ ]:
def for_profiler(): # use just code inside the function when not using the profiler
    for source_col in ['source_0','source_1','source_2','source_3','source_4']:
        # this is just on 5 first entries, as a test.
        df2015_with_info.loc[:5,source_col] = df2015_with_info.loc[:5,source_col].apply(scraper)
    return

%lprun -f scraper for_profiler()

In [82]:
data_with_2 = data_with.copy()

def for_profiler(): # use just code inside the function when not using the profiler
    for source_col in ['source_0','source_1','source_2','source_3','source_4']:
        for ind, name in zip(data_with_2.index,data_with_2.name):
            data_with_2.loc[ind,source_col] = extractor(data_with_2.loc[ind,source_col],name)
    return

%lprun -f extractor for_profiler()

---

In [2]:
from pandas.io import sql

data_with15 = sql.read_sql("""SELECT * FROM data2015_with_html;""", engine)
data_with16 = sql.read_sql("""SELECT * FROM data2016_with_html;""", engine)

In [3]:
data_with15.drop(['index','wp_index','city','is_geocoding_exact','is_officer_charged','threat_level_display', 'weapon'],axis=1, inplace=True)
data_with16.drop(['index','level_0','officers_identified', 'weapon'], axis=1, inplace=True)
data_with16["source_4"] = None

In [4]:
print data_with15.shape, data_with16.shape

(94, 21) (7, 21)


In [5]:
data_with_html = pd.concat([data_with15,data_with16], ignore_index=True)

In [ ]:
data_with_2 = data_with_html.copy() # for use with extractor() funtion

# this uses just the name of the victim. Trying with other info could/should increase yield
for source_col in ['source_0','source_1','source_2','source_3','source_4']:
    for ind, name in zip(data_with_2.index,data_with_2.name):
        data_with_2.loc[ind,source_col] = extractor(data_with_2.loc[ind,source_col],name)

In [ ]:
data_with_articles = data_with_2

In [16]:
data_with_2[['source_0','source_1','source_2','source_3','source_4']].head()

,source_0,source_1,source_2,source_3,source_4
0,No match.,"\n John Paul Quintero, 23, was shot by police outside this residence on North Oliver on Saturday after a disturbance was reported to 911 operators. (Jan. 4, 2015)\n \n John Paul Quintero, 23, was shot by police outside this residence on North Oliver on Saturday after a disturbance was reported to 911 operators. (Jan. 4, 2015)\n \n \n The frantic call came in to 911 early Saturday evening.\n \n \n “Like my nephew’s at my daughter’s house … he has a knife. He’s drunk, he’s high … his daughter’s there. My son-in-law’s there trying to hold him back. Somebody’s gonna end up getting hurt.”\n \n \n The caller, identified in the 911 transcript as Gabriela Torres, was telling an emergency dispatcher about her nephew, John Paul Quintero, who Torres said was armed with a knife at her daughter’s house in the 500 block of North Oliver shortly before 6:45 p.m. Saturday.\n \n \n ...","No html, or is NaN value.","No html, or is NaN value.","No html, or is NaN value."
1,"\n that Autumn Steele was “beating up on” her husband.\n \n \n When Hill “ran over to break up the fight, the dog ran over to the cop, kind of in a playful way (and) jumped up on the cop’s back,” the witness said. “I don’t believe the cop ever even seen the dog until he was on his back. He spun around trying to deal with the dog. He pulled his weapon and he was firing at the dog.\n \n \n “I believe he shot one before he went down, but I’m not positive about that,” the witness continued. “It happened so quickly, so both shots might have happened while he was falling down, during which he hits the lady and she went down.”\n \n \n The Iowa Division of Criminal Investigation, which is overseeing the investigation into the shooting, said its preliminary findings support a tragic accident. The DCI confirmed that Hill fired his weapon twice.\n ...","\n \n On Jan. 6, Autumn Steele, 34, was fatally shot outside her home by a Burlington police officer investigating a domestic disturbance. The officer was startled by the couple’s German shepherd, Sammy, which ran outside. The officer fired several shots at the dog when he fell. One of the shots missed the dog and hit Steele in the chest.\n \n \n \n “I’m just very thankful,” said Gina Colbert, mother of Autumn Steele. “We came here from Georgia (for the hearing) because we never thought Sammy was vicious. I just don’t know what the next step is. I don’t know if we can take him back to Georgia with us or leave him here with Gabe. We will just have to wait and see. But I do know enough is enough. We are a family in pain. My daughter is dead, and my 7-year-old grandson is having a very difficult time (in Georgia) accepting his mother’s death.”\n \n","\n Members of the Burlington Police Department stand near the scene of a shooting Jan. 6, 2015, in Burlington. Officer Jesse Hill accidentally shot and killed Autumn Steele while firing his gun at her attacking German shepherd. Hill could be back on the job as soon as Friday.\n \n \n Officer Jesse Hill was cleared late last month of criminal charges in the fatal shooting of 34-year-old Autumn Steele of Burlington after he responded to a domestic dispute Jan. 6 between Steele and her husband, Gabriel Steele, over their 4-year-old child.\n \n \n \n Twelve seconds of edited footage from Hill's body camera, released this week by the Iowa Division of Criminal Investigation, shows how the incident quickly spun out of control, abruptly ending with the officer firing his weapon twice and unintentionally shooting Autumn Steele in the chest.\n \n \n \n An autopsy determined that Autumn Steele died from a gunshot...","No html, or is NaN value.","No html, or is NaN value."
2,"\n \n LITTLE ROCK, Ark. (AP) - Authorities are investigating the fatal shooting of a robbery suspect who they say died after a struggle with a Union County sheriff’s deputy.\n \n \n The Union County Sheriff says a deputy arrived at a pharmacy in Strong after an alarm triggered 

In [12]:
pb_data = []
for col in data_with_2.columns:
    if data_with_2[col].dtype == object:
        for ind, value in zip(data_with_2.index,data_with_2[col]):
            if value == None:
                continue
            try :
                data_with_2.loc[ind,col] = value.encode()
            except UnicodeEncodeError:
                pb_data.append([(ind,col),value])

In [14]:
len(pb_data)

86

In [17]:
pb_data

[[(20, u'description'),
  u'A Volusia County sheriff\u2019s deputy was part of a squad serving a search warrant for narcotics at the home when he shot Cruice in the face.'],
 [(36, u'description'),
  u'Felix, who was diagnosed with schizophrenia, fled from New York City detectives who wanted to interview him about a robbery. He grabbed an officer\u2019s radio and struck a detective on the head.'],
 [(1, u'source_0'),
  u'\n               that Autumn Steele was \u201cbeating up on\u201d her husband.\n               \n               \n               When Hill \u201cran over to break up the fight, the dog ran over to the cop, kind of in a playful way (and) jumped up on the cop\u2019s back,\u201d the witness said. \u201cI don\u2019t believe the cop ever even seen the dog until he was on his back. He spun around trying to deal with the dog. He pulled his weapon and he was firing at the dog.\n               \n               \n               \u201cI believe he shot one before he went down, bu

In [24]:
set([x[0][1] for x in pb_data])

{u'description',
 u'source_0',
 u'source_1',
 u'source_2',
 u'source_3',
 u'source_4'}

In [16]:
data_with_2.iloc[:,:16].to_csv("test_unicode.csv")

UnicodeEncodeError: 'ascii' codec can't encode character u'\u2019' in position 24: ordinal not in range(128)